In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
# Import data vis tool
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

## General EDA

In [ ]:
# Load dataset
data = pd.read_csv("/kaggle/input/fraud-detection/fraudTrain.csv")

data.head()

In [ ]:
# split dataset for further processing
fraud = data[data['is_fraud']==1]
normal = data[data['is_fraud']==0]

### There is no null data

In [ ]:
data.isnull().sum().sum()

### Examine the index list

In [ ]:
# data.columns
numeric_features = data.select_dtypes(include=[np.number])

numeric_features.columns

### Correlation Heat Map

In [ ]:
f , ax = plt.subplots(figsize = (14,12))
plt.title('Correlation of Numeric Features',y=1,size=16)
sns.heatmap(data.corr(),square = True,  vmax=0.8)

In [ ]:
cols = data.corr().nlargest(5,'is_fraud')['is_fraud']
print(cols)

## Amount && is_fraud

In [ ]:
# See overall ststs
pd.concat([fraud.amt.describe(), normal.amt.describe()], axis=1)

In [ ]:
plt.figure(figsize=(30, 10))

print()
plt.subplot(2, 1, 1)
subfraud = fraud[fraud.amt<500]
subfraud.amt.hist(bins=400,weights=np.ones(len(subfraud.amt)) / len(subfraud.amt))
plt.title("Fraudulant Transaction Amount Distribution")
plt.legend()
plt.xlim([-10,400])
plt.ylim([0,0.1])

plt.subplot(2, 1, 2)
subnormal = normal[normal.amt<400]
subnormal.amt.hist(bins=500, weights=np.ones(len(subnormal.amt)) / len(subnormal.amt))
plt.title("Normal Transaction Amount Distribution")
plt.legend()
plt.xlim([-10,400])
plt.ylim([0,0.1])

## Time && is_fraud

In [ ]:
# plot hist 
plt.figure(figsize=(20, 10))

plt.subplot(2, 1, 1)
fraud_24time = fraud.copy()
fraud_24time.unix_time = fraud_24time.unix_time % 86400 #(86400s per 24h)
fraud_24time.unix_time /= (86400/24)
fraud_24time.unix_time.hist(bins=50,label="Fraudulant Transaction",weights=np.ones(len(fraud_24time)) / len(fraud_24time))
plt.xlim([0,24])
plt.ylim([0,0.15])
plt.xlabel("")
plt.legend()

plt.subplot(2, 1, 2)
normal_24time = normal.copy()
normal_24time.unix_time  = normal_24time.unix_time  % 86400 #(86400s per 24h)
normal_24time.unix_time  /= (86400/24)
normal_24time.unix_time .hist(bins=50,label="Non Fraudulant Transaction",weights=np.ones(len(normal_24time)) / len(normal_24time))
plt.xlim([0,24])
plt.ylim([0,0.15])
plt.legend()

## Summary

Genral EDA
* **!!!Concern:** This is a simulated dataset, which may not truly reflect the reality.
* Exist some duplicate data (high correlation nearly +-1) e.g. merch_long & long, merch_lat & lat, long & zip unnamed_0 & unix_time
* Some categorical data need to be transformed into numerical one, e.g.(gender 0:female 1:male)
* There is no null data or empty data. Because this is simulated data, it is not unexpected.
* **Amt** & **isFraud** has high correlation. -> Amt can be quite critical and useful to determine whether it's a fraud.


 Take a further look at Amt
* It's apparent that both Fraud Trans Amount and Normal Trans Amount don't follow normal distrubution.
* We can always see two peaks pattern in Fraud Trans Amount distribution.

What about time
* It's more obvious in this dataset, we can find the probability of fraud in a certain period of time is particularly high. (22pm-4am)